In [ ]:
import os
import sys
import itertools

# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import scipy.stats

import matplotlib.pyplot as plt

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_visible_devices([gpus[1]], 'GPU')

sys.path.append("/nfs/scistore12/chlgrp/vvolhejn/smooth")

In [ ]:
%load_ext autoreload
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%autoreload 1

In [ ]:
def saw(x):
    x = (x+1) % 2
    return np.min([x, 2-x], axis=0)

x_train = np.linspace(-2, 2, 5, dtype=np.float32).reshape((-1, 1))
y_train = saw(x_train)

x_test = np.linspace(-3, 3, 61, dtype=np.float32).reshape((-1, 1))
y_test = saw(x_test)

dataset = smooth.datasets.Dataset(x_train, y_train, x_test, y_test)

In [ ]:
np.logspace(np.log10(3.), np.log10(300.), 20)

In [ ]:
# init_scales = [300., 100., 30., 10., 3.]
init_scales = np.logspace(np.log10(10.), np.log10(1000.), 20)
models = dict()

for init_scale in init_scales:
    models[init_scale] = smooth.model.train_shallow_relu(
        dataset,
        learning_rate=0.1 / init_scale,
        init_scale=init_scale,
        epochs=10000,
        hidden_size=100,
        batch_size=len(x_train),
        verbose=0,
        callbacks=[
            tf.keras.callbacks.EarlyStopping("loss", min_delta=1e-5, patience=100),
            smooth.callbacks.Tqdm(verbose=0),
            tf.keras.callbacks.TerminateOnNaN()
        ],
        train_val_split=1.0,
    )

    smooth.analysis.plot_shallow_relu(models[init_scale], dataset)

In [ ]:
measures = dict()

for init_scale, model in tqdm.notebook.tqdm(models.items()):
    measures[init_scale] = smooth.measures.get_measures(model, x_test, y_test)

In [ ]:
measure_names = measures[x[0]].keys()

x = sorted(measures.keys())
yd = dict(zip(measure_names,[[] for _ in range(len(measure_names))]))

for init_scale in x:
    for k, v in measures[init_scale].items():
        yd[k].append(v)

for measure_name in measure_names:
    plt.plot(x, yd[measure_name])
    plt.title(measure_name)
    plt.xscale("log")
    plt.show()

In [ ]:
smooth.measures.get_measures(models[3.0], x_test, y_test)

In [ ]:
smooth.analysis.plot_shallow_relu(_model, dataset)

In [ ]:
measures